In [1]:
# Parameters
# papermill: parameters
# tags: [parameters]

mode = None
input_path = None
output_path = None


In [2]:
# Parameters
mode = "fake"
input_path = ["C:\\store\\git\\km-stat-activity\\parquet_dataset\\date=2025-04-21\\profile_guid=011d3524-6fce-4184-bfa3-9707e8c6f18c\\part.6.parquet"]
output_path = "C:\\store\\git\\km-stat-activity\\processed\\fake\\profile_guid=011d3524-6fce-4184-bfa3-9707e8c6f18c\\2025-04-21-processed.csv"


In [3]:

import math
import pandas as pd
from pathlib import Path
import os

# Papermill’den gelen parametreler:
#   mode: "fake" veya "real"
#   input_path: ya bir liste (parquet yolları) ya da bir string (csv yolu)
#   output_path: yazılacak csv’nin tam yolu


if isinstance(input_path, list):
    # Fake: parquet dosyalarını birleştir
    paths = [Path(p) for p in input_path]
    df = pd.concat([pd.read_parquet(p, engine="pyarrow") for p in paths],
                   ignore_index=True)
else:
    # Real: tek CSV oku
    df = pd.read_csv(str(input_path),
                     converters={"x": pd.eval, "y": pd.eval},
                     parse_dates=['start_date_time','end_date_time'])


# 2) Artık df hazır, geri kalan tüm hücreleri olduğu gibi çalıştırabilirsiniz.


# NaN'leri filtrele
df = df[df['x'].notna() & df['y'].notna()].copy()

# Küme olasılığına karşı güvenli parser
def safe_parse_list(s):
    try:
        parsed = ast.literal_eval(s)
        return list(parsed)
    except Exception:
        return []

df['x'] = df['x'].apply(safe_parse_list)
df['y'] = df['y'].apply(safe_parse_list)

def count_direction_changes(arr):
    count = 0
    for i in range(2, len(arr)):
        diff1 = arr[i-1] - arr[i-2]
        diff2 = arr[i] - arr[i-1]
        if diff1 * diff2 < 0:
            count += 1
    return count
df['x_direction_changes'] = df['x'].apply(count_direction_changes)
df['y_direction_changes'] = df['y'].apply(count_direction_changes)

df_result = df[["x_direction_changes", "y_direction_changes"]]

# df_result zaten hesaplandı varsayılıyor
if os.path.exists(output_path):
    df_existing = pd.read_csv(output_path)
    df_merged = pd.concat(
        [df_existing.reset_index(drop=True), df_result.reset_index(drop=True)],
        axis=1
    )
else:
    df_merged = df_result

df_merged.to_csv(output_path, index=False)
print(f"✔️ Saved CSV with directional_changes: {output_path}")


✔️ Saved CSV with directional_changes: C:\store\git\km-stat-activity\processed\fake\profile_guid=011d3524-6fce-4184-bfa3-9707e8c6f18c\2025-04-21-processed.csv
